# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\Mario\python projects\Data Engineering\Project 2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra creation and queries. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#  Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#  Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data



#### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



    






based on the needed query, we will add the following columns
sessionId,
itemInSession,
artist,
song,
length,
and will make the partition key on session id and itemInSession to be unique and no need for cluster column since no custome sorting is required


In [10]:

query = """CREATE TABLE IF NOT EXISTS sparkify_session_item (
sessionId int,
itemInSession int,
artist text,
song text,
length float,
PRIMARY KEY (sessionId, itemInSession));"""
try:
    session.execute(query)
except Exception as e:
    print(e)     

                    

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into sparkify_session_item (sessionId ,itemInSession , artist , song ,length )"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [23]:

query = """select artist, song,length
           from sparkify_session_item
           WHERE  sessionId = 338 and itemInSession = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"artist is {row.artist}, \n song is {row.song},\n song length is {row.length/60} mins")

artist is Faithless, 
 song is Music Matters (Mark Knight Dub),
 song length is 8.255121866861979 mins


from the above query, we can see that all the songs and their respective artists that the users played during session No 338 and the item in seesion is 4
which is  "Music Matters (Mark Knight Dub) by Faithless"

#### Query 2. retierve only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

based on the needed query, we will add the following columns
userid,
sessionid,
itemInSession,
artist,
song,
firstName,
lastName,
and will make the partition key on userid and session id to be unique and clustering column on iteminsession to satisfy the sorting requirements


In [40]:


query = """CREATE TABLE IF NOT EXISTS sparkify_song_user (
userid int,
sessionid int,
itemInSession int,
artist text,
song text,
firstName TEXT,
lastName TEXT,
PRIMARY KEY ((userid, sessionid),itemInSession));"""
try:

    session.execute(query)
except Exception as e:
    print(e)     


                    

In [41]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO sparkify_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [44]:

query = """select artist, song,firstName,lastName
           from sparkify_song_user
           WHERE  userid = 10 and sessionid = 182
           ORDER BY itemInSession;"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"artist is {row.artist}, \n song is {row.song},\n username is {row.firstname} {row.lastname}")

artist is Down To The Bone, 
 song is Keep On Keepin' On,
 username is Sylvie Cruz
artist is Three Drives, 
 song is Greece 2000,
 username is Sylvie Cruz
artist is Sebastien Tellier, 
 song is Kilometer,
 username is Sylvie Cruz
artist is Lonnie Gordon, 
 song is Catch You Baby (Steve Pitron & Max Sanna Radio Edit),
 username is Sylvie Cruz


from the above query, we can see that the songs and their respective artists that user id No 10 listened to in session No 182

#### Query 3. retrieve every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

based on the needed query, we will add the following columns
song,
firstname,
lastname,
userid,
and will make the partition key on song, firstname,lastname and userid to be unique and no need for cluster column since no custome sorting is required



In [18]:

query = """CREATE TABLE IF NOT EXISTS sparkify_username_song (
song text,
firstname TEXT,
lastname TEXT,
userid  INT,
PRIMARY KEY (song, firstname,lastname,userid ));"""
try:
    session.execute(query)
except Exception as e:
    print(e)     


In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO sparkify_username_song (song, firstname, lastname,userid)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [49]:

query="""select firstname,lastname from sparkify_username_song where song='All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f"username is {row.firstname} {row.lastname}")                   

username is Jacqueline Lynch
username is Sara Johnson
username is Tegan Levine


we can see from the above query, that only 3 users has listened to 'All Hands Against His Own' and they are Jacqueline Lynch, Sara Johnson, and Tegan Levine.

### Drop the tables before closing out the sessions

In [51]:
## Drop the table before closing out the sessions
query0="""Drop Table sparkify_session_item"""
query1="""Drop Table sparkify_song_user"""
query2="""Drop Table sparkify_username_song"""

try:

    session.execute(query0)
    session.execute(query1)
    session.execute(query2)
except Exception as e:
    print(e)     


Error from server: code=2200 [Invalid query] message="unconfigured table sparkify_session_item"


### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()